Applying convolution to Text -Chapter 9

Detecting sarcasm in text using CNNs

In [4]:
import pandas as pd
import numpy as np
import re
import json
import gensim
import math
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import KeyedVectors
import keras
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Dense, Dropout, Conv1D, GlobalMaxPooling1D
import h5py


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
import json

def parse_data(file):
    for line in open(file, 'r'):
        try:
            yield json.loads(line)
        except json.JSONDecodeError as e:
            print(f"Error in line: {line}")
            raise e

try:
    data = list(parse_data('/content/Sarcasm_Headlines_Dataset_v2.json'))
    df = pd.DataFrame(data)
except json.JSONDecodeError as e:
    print(e)


In [6]:
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [7]:
df.shape

(28619, 3)

In [8]:
df.drop(columns="article_link")

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...
...,...,...
28614,1,jews to celebrate rosh hashasha or something
28615,1,internal affairs investigator disappointed con...
28616,0,the most beautiful acceptance speech this week...
28617,1,mars probe destroyed by orbiting spielberg-gat...


In [9]:
classes = np.unique(np.array(df['is_sarcastic']))
classes

array([0, 1])

In [17]:
def text_clean(corpus):
    '''
    Purpose : Function to keep only alphabets, digits and certain words (punctuations, qmarks, tabs etc. removed)

    Input : Takes a text corpus, 'corpus' to be cleaned along with a list of words, 'keep_list', which have to be retained
            even after the cleaning process

    Output : Returns the cleaned text corpus

    '''
    cleaned_corpus = []  # Initialize as an empty list
    for row in corpus:
        qs = []
        for word in row.split():
            p1 = re.sub(pattern='[^a-zA-Z0-9]',repl=' ',string=word)
            p1 = p1.lower()
            qs.append(p1)
        cleaned_corpus.append(' '.join(qs))  # Append the cleaned string to the list
    return pd.Series(cleaned_corpus)  # Convert the list to a Series before returning


In [18]:
def stopwords_removal(corpus):
    stop = set(stopwords.words('english'))
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

In [19]:
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

In [20]:
def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

In [21]:
def preprocess(corpus, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):

    '''
    Purpose : Function to perform all pre-processing tasks (cleaning, stemming, lemmatization, stopwords removal etc.)

    Input :
    'corpus' - Text corpus on which pre-processing tasks will be performed

    'cleaning', 'stemming', 'lemmatization', 'remove_stopwords' - Boolean variables indicating whether a particular task should
                                                                  be performed or not
    'stem_type' - Choose between Porter stemmer or Snowball(Porter2) stemmer. Default is "None", which corresponds to Porter
                  Stemmer. 'snowball' corresponds to Snowball Stemmer

    Note : Either stemming or lemmatization should be used. There's no benefit of using both of them together

    Output : Returns the processed text corpus

    '''
    if cleaning == True:
        corpus = text_clean(corpus)

    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]

    if lemmatization == True:
        corpus = lemmatize(corpus)


    if stemming == True:
        corpus = stem(corpus, stem_type)

    corpus = [' '.join(x) for x in corpus]


    return corpus

In [22]:
df['headline']

0        thirtysomething scientists unveil doomsday clo...
1        dem rep. totally nails why congress is falling...
2        eat your veggies: 9 deliciously different recipes
3        inclement weather prevents liar from getting t...
4        mother comes pretty close to using word 'strea...
                               ...                        
28614         jews to celebrate rosh hashasha or something
28615    internal affairs investigator disappointed con...
28616    the most beautiful acceptance speech this week...
28617    mars probe destroyed by orbiting spielberg-gat...
28618                   dad clarifies this not a food stop
Name: headline, Length: 28619, dtype: object

In [23]:
headlines = preprocess(df['headline'], lemmatization = True, remove_stopwords = True)

In [24]:
headlines[0:5]

['thirtysomething scientists unveil doomsday clock hair loss',
 'dem rep totally nail congress fall short gender racial equality',
 'eat veggies 9 deliciously different recipes',
 'inclement weather prevent liar get work',
 'mother come pretty close use word stream correctly']

Loading Word2vec model

In [25]:
#!pip install gensim
import gensim
from gensim.models import KeyedVectors
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [30]:
model=KeyedVectors.load_word2vec_format('/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz',binary=True)

Defining model Parameters


In [31]:
MAX_LENGTH = 10
VECTOR_SIZE = 300

Data vectorization and standarization

we will be sending the word vectors, each of size 300, to
our CNN model. However, for that, our data dimensions should be
standardized. Since our data is from headlines, we will standardize them so that
they have a length of 10. If any of the headlines contain more than 10 characters,
they will be subsampled to keep the word embeddings for the first 10 characters.
If any of the headlines contain less than 10 characters, we will pad them so that
they have vectors with values of 0.


In [32]:
def vectorize_data(data, model):
    vectors = []
    padding_vector = [0.0] * VECTOR_SIZE

    for data_point in data:
        data_point_vectors = []
        count = 0

        tokens = data_point.split()

        for token in tokens:
            if count >= MAX_LENGTH:
                break
            if token in model.key_to_index:  # Changed from model.wv.key_to_index
                data_point_vectors.append(model[token])  # Changed from model.wv[token]
            count += 1

        # Padding the remaining vectors if the data point is shorter than MAX_LENGTH
        data_point_vectors += [padding_vector] * (MAX_LENGTH - len(data_point_vectors))

        vectors.append(data_point_vectors)

    return vectors


In [33]:
vectorized_headlines = vectorize_data(headlines,model)

 Let's add a small validation to ensure that the 10 vectors are present for each
headline, as defined in the MAX_LENGTH parameter:


In [34]:
for i, vec in enumerate(vectorized_headlines):
 if len(vec) != MAX_LENGTH:
  print(i)


In [35]:
len(vectorized_headlines[i])

10

In [36]:
train_div = math.floor(0.7 * len(vectorized_headlines))
train_div

20033

In [37]:
X_train = vectorized_headlines[:train_div]
y_train = df['is_sarcastic'][:train_div]
X_test = vectorized_headlines[train_div:]
y_test = df['is_sarcastic'][train_div:]

print('The size of X_train is:', len(X_train), '\nThe size of y_train is:', len(y_train),
      '\nThe size of X_test is:', len(X_test), '\nThe size of y_test is:', len(y_test))

The size of X_train is: 20033 
The size of y_train is: 20033 
The size of X_test is: 8586 
The size of y_test is: 8586


In [38]:

X_train = np.reshape(X_train, (len(X_train), MAX_LENGTH, VECTOR_SIZE))
X_test = np.reshape(X_test, (len(X_test), MAX_LENGTH, VECTOR_SIZE))
y_train = np.array(y_train)
y_test = np.array(y_test)


Defining Neural Network Model Parameters

In [39]:
FILTERS=8
KERNEL_SIZE=3
HIDDEN_LAYER_1_NODES=10
HIDDEN_LAYER_2_NODES=5
DROPOUT_PROB=0.35
NUM_EPOCHS=10
BATCH_SIZE=50

In [40]:
model = Sequential()

model.add(Conv1D(FILTERS,
                 KERNEL_SIZE,
                 padding='same',
                 strides=1,
                 activation='relu',
                 input_shape = (MAX_LENGTH, VECTOR_SIZE)))
model.add(GlobalMaxPooling1D())
model.add(Dense(HIDDEN_LAYER_1_NODES, activation='relu'))
model.add(Dropout(DROPOUT_PROB))
model.add(Dense(HIDDEN_LAYER_2_NODES, activation='relu'))
model.add(Dropout(DROPOUT_PROB))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 10, 8)             7208      
                                                                 
 global_max_pooling1d (Glob  (None, 8)                 0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 10)                90        
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dropout_1 (Dropout)         (None, 5)                 0         
                                                        

In [41]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [42]:
training_history = model.fit(X_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/10
401/401 [==============================] - 4s 6ms/step - loss: 0.6489 - accuracy: 0.6477
Epoch 2/10
401/401 [==============================] - 3s 7ms/step - loss: 0.5704 - accuracy: 0.7401
Epoch 3/10
401/401 [==============================] - 2s 5ms/step - loss: 0.5175 - accuracy: 0.7738
Epoch 4/10
401/401 [==============================] - 2s 5ms/step - loss: 0.4834 - accuracy: 0.7882
Epoch 5/10
401/401 [==============================] - 2s 5ms/step - loss: 0.4588 - accuracy: 0.8082
Epoch 6/10
401/401 [==============================] - 2s 5ms/step - loss: 0.4309 - accuracy: 0.8213
Epoch 7/10
401/401 [==============================] - 3s 6ms/step - loss: 0.4056 - accuracy: 0.8342
Epoch 8/10
401/401 [==============================] - 3s 8ms/step - loss: 0.3957 - accuracy: 0.8401
Epoch 9/10
401/401 [==============================] - 2s 5ms/step - loss: 0.3758 - accuracy: 0.8487
Epoch 10/10
401/401 [==============================] - 2s 5ms/step - loss: 0.3592 - accuracy: 0.8567

Model Evaluation

In [43]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7552


**Chapter 10**


Building a text gernerator

In [46]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding


In [47]:
data = pd.read_csv('/content/hotel_data.csv')


In [48]:
data.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
0,Hardasji Ki Magri,Udaipur,India,2016-06-21,{{facility}},|Zion Home Stay is located in a city that sets...,1 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,78ddf880bd7937d384ff278cc5b39d6e
1,Near Nai Gaon,Udaipur,India,2016-06-21,{{facility}},| Araliayas Resorts is a 3 star hotel located ...,3 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,9f9f9cbb2f7df8089b63d5cdeb257944
2,Near Bagore Ki Haveli,Udaipur,India,2016-06-21,{{facility}},|A 2 star property is located at 24 km from Ma...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,NaN,makemytrip,NaN,NaN,b314bb7fa8bfb1ed306f517be21d729e
3,Dabok,Udaipur,India,2016-06-21,Airport Transfer|Car rental|Conference Hall|Cu...,|SNP House Airport Hotel And Restaurant is loa...,1 star,//imghtlak.mmtcdn.com/images/hotels/2014071815...,NaN,no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,`standard,/5,NaN,makemytrip,NaN,NaN,e6f5bb3c2d76a78d978b9ceb0e31ec56
4,East Udaipur,Udaipur,India,2016-06-21,{{facility}},| Hotel Pichola Haveli is situated in the beau...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,3.7,makemytrip,NaN,NaN,63072c301427b6ca450d31eea127bcf0


In [49]:
data.city.value_counts()

city
NewDelhiAndNCR    1163
Goa               1106
Mumbai             539
Jaipur             501
Bangalore          477
                  ... 
Bhojpur              1
Khatu                1
Anandpur Sahib       1
Sravasti             1
Orai                 1
Name: count, Length: 759, dtype: int64

In [50]:
array = ['Mumbai']
data = data.loc[data['city'].isin(array)]


In [51]:
data.head(5)

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
294,Charai,Mumbai,India,2016-08-28,Doctor on Call|Front desk|Laundry Service|Park...,"Nestled in Mumbai, a city with strong historic...",3,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Ac Superior Room|A/c Standard Rooms Double Occ...,NaN,NaN,makemytrip,Maharashtra,NaN,d78fae90ef2e1b5c2dfd547c61763a25
309,Andheri (East),Mumbai,India,2016-08-28,Air Conditioned|Airport Transfer|Conference Ha...,3 km from Chhatrapati Shivaji International Ai...,2,NaN,Bathroom Toiletries|Daily Newspaper|Kitchenett...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Executive|Deluxe,NaN,NaN,makemytrip,Maharasta,Location:3.2/5 | Hospitality:3.1/5 | Facilitie...,030865f741982beb373efddecdc6d6c3
321,Khar,Mumbai,India,2016-08-28,Airport/Rlwy Stn Transfer|Bar|Conference Hall|...,Location Hotel Royal Garden is situated on Juh...,3,NaN,Electronic Safe|Bathroom Toiletries|Daily News...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Superior Executive,NaN,NaN,makemytrip,Maharashtra,Location:4.5/5 | Hospitality:3.4/5 | Facilitie...,a1ced509350038775a7700ec67796bc2
334,Andheri (East),Mumbai,India,2016-08-28,24 Hour Check in-Icon|24 hour reception|24 hou...,City Guest House is a beautiful property locat...,2,NaN,Bathroom Toiletries|Hot/cold Water|Attached Ba...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Standard Room|Deluxe Room|Triple Deluxe Room|S...,NaN,NaN,makemytrip,Maharashtra,Location:4.3/5 | Hospitality:3.8/5 | Facilitie...,f2820ae7707668ed6906bb227921f720
1238,Andheri (East),Mumbai,NaN,2016-08-22,24 Hour Check in-Icon|24 hour reception|24 hou...,Sai Residency Hotel is situated in the City of...,2,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-22 22:10:53 +0000,2016-08-22 22:10:53 +0000,Deluxe Dbl Air Cooled,NaN,NaN,makemytrip,MAHARASHTRA,NaN,b4af24952027ffbcd85a91bb6fe23f5d


In [52]:
data = data.hotel_overview
data = data.dropna()


Removing stopwords

In [53]:
stop = set(stopwords.words('english'))
def stopwords_removal(data_point):
 data = [x for x in data_point.split() if x not in stop]
 return data

In [55]:
def clean_data(data):
 cleaned_data = []
 all_unique_words_in_each_description = []
 for entry in data:
  entry = re.sub(pattern='[^a-zA-Z]',repl=' ',string = entry)
  entry = re.sub(r'\b\w{0,1}\b', repl=' ',string = entry)
  entry = entry.lower()
  entry = stopwords_removal(entry)
  cleaned_data.append(entry)
  unique = list(set(entry))
  all_unique_words_in_each_description.extend(unique)
  return cleaned_data, all_unique_words_in_each_description

Figuring out unique words which will be our vocabulary

In [56]:
def unique_words(data):
 unique_words = set(all_unique_words_in_each_description)
 return unique_words, len(unique_words)

In [57]:
cleaned_data, all_unique_words_in_each_description = \
 clean_data(data)
unique_words, length_of_unique_words = \
 unique_words(all_unique_words_in_each_description)
